In [1]:
type Node{T}
    left::Nullable{Node{T}}
    right::Nullable{Node{T}}
    value::T
end

Node{T}(v::T) = Node(Nullable{Node{T}}(), Nullable{Node{T}}(), v)

Node{T}

In [1]:
n1 = Node(Nullable(Node(2)), Nullable(Node(3)), 10)

LoadError: LoadError: UndefVarError: Node not defined
while loading In[1], in expression starting on line 1

In [12]:
left(n::Node) = isnull(n.left) ? Void(): get(n.left)
right(n::Node) = isnull(n.right) ? Void(): get(n.right)

right (generic function with 1 method)

In [91]:
function search{T}(graph::Node{T}, number)
    for i = 1:10
        if graph.value == number
            return number
        else
            try
                graph = graph.left.value
            catch
                return 0
            end
        end
    end
end
    

search (generic function with 1 method)

In [1]:
include("SymmetricMatrix.jl")
include("Test.jl")
using Test

In [2]:
include("SymmetricMatrix.jl")
using SymmetricMatrix
using NullableArrays
importall SymmetricMatrix

In [3]:
using Iterators
function genstensor(T::Type, dims::Int, l::Int, seed::Int = 1234)
    srand(seed)
    ret = zeros(T, fill(l, dims)...)
    indices = collect(product(fill(collect(1:l), dims)...))
    elements = Array{T,1}(randn(l^dims÷dims))
    r = 1
    for i in indices 
        if issorted(i)          
            for k in collect(permutations(i))
                ret[k...] = elements[r]
            end
            r += 1
        end
    end
    ret
end

genstensor (generic function with 2 methods)

In [13]:
l = 100
st = genstensor(Float64, 4,l);
st1 = genstensor(Float64, 4,l, 1233);
c = randn(l,l-10);

In [14]:
bs = (convert(BoxStructure{Float64}, st, 10)) ;
@time bs1 = (convert(BoxStructure{Float64}, st1, 10)) ;

  3.941880 seconds (320.93 k allocations: 9.016 GB, 4.65% gc time)


In [15]:
@time bs+bs1;
@time bs+2;
@time st + st1;
@time st + 2;

  0.147497 seconds (326.91 k allocations: 77.428 MB, 32.81% gc time)
  0.085748 seconds (306.89 k allocations: 76.620 MB)
  0.206162 seconds (7 allocations: 762.940 MB, 18.99% gc time)
  0.165327 seconds (7 allocations: 762.940 MB, 25.14% gc time)


In [16]:
@time convert(Array{Float64}, bs);
@time vec(bs);
@time vec(st);

  0.849222 seconds (1.22 M allocations: 1.541 GB, 13.24% gc time)
  0.810225 seconds (1.22 M allocations: 1.541 GB, 9.61% gc time)
  0.000001 seconds (6 allocations: 240 bytes)


In [17]:
@time bs.*bs1;
@time bs+bs1;
@time bs./bs1;
@time st.*st1;
@time st+st1;
@time st./st1;

  0.108844 seconds (331.20 k allocations: 77.689 MB, 3.25% gc time)
  0.143781 seconds (326.91 k allocations: 77.428 MB, 32.65% gc time)
  0.148940 seconds (331.20 k allocations: 77.689 MB, 6.15% gc time)
  0.288442 seconds (13 allocations: 762.940 MB, 13.97% gc time)
  0.209621 seconds (7 allocations: 762.940 MB, 20.73% gc time)
  0.796712 seconds (13 allocations: 762.940 MB, 4.31% gc time)


In [18]:
@time modemult(bs, c', 4)
@time Tensors.modemult(st,c',4);

 16.540331 seconds (20.96 M allocations: 35.310 GB, 9.51% gc time)
  2.585921 seconds (177 allocations: 2.086 GB, 2.35% gc time)
